In [ ]:
from math import *
from utils import *
import ROOT
import os

### Setting the image path 

In [ ]:
from datetime import date
#path of folder to save the images
imgPath = './images/paper/%s/'%(date.today().strftime('%Y-%m-%d'))

### Reading trees and selecting branches

In [ ]:
# getting signal and background trees
# signal
backgroundPath = './data/background/'
signalPath = './data/VBF_MDSimp_EWKExcluded/'

fs = [] #Signal files
sNames = [] #signal names
for f in os.listdir(signalPath):
    if os.path.isfile(signalPath + f) and '.root' in f:
        fs.append(ROOT.TFile(signalPath + f))
        sNames.append(f)
        if not os.path.exists(imgPath):
            os.makedirs(imgPath)
    
        
#Signal trees
s = [f.Get("Delphes") for f in fs]

# background
b = ROOT.TChain("Delphes")
b.Add(backgroundPath + "santiago_run_02.root") #no
b.Add(backgroundPath + "santiago_run_05.root") #no
b.Add(backgroundPath + "santiago_run_07.root") #no

# number of signal and background events 
nb = int(b.GetEntries())
ns = [int(k.GetEntries()) for k in s]

# number of bins to book histograms
nbBins = int(sqrt(nb))+1
nsBins = [int(sqrt(n))+1 for n in ns] 

# reading trees and selecting branches
treeB = ROOT.ExRootTreeReader(b)
treeS = [ROOT.ExRootTreeReader(t) for t in s]

jetB = treeB.UseBranch("Jet")
jetS = [tree.UseBranch("Jet") for tree in treeS]

metB = treeB.UseBranch("MissingET")
metS = [tree.UseBranch("MissingET") for tree in treeS]

# number of jets to use
numJets = 5

### Book histograms

In [ ]:
#---------------------------
#           signal
#---------------------------

# #Delta Eta for the two jets with highest invariant mass
# hsDeltaEtaMaxIM = [ROOT.TH1F("hsDeltaEtaM_{0}".format(j), "Delta_Eta_M_{0}".format(j), 50,0,10) 
#                    for j in range(len(s))]
# hsMass = [ROOT.TH1F("hSMass_{0}".format(j), "M(j1,j2)_{0}".format(j), 65,0,4000)
#          for j in range(len(s))]


hsHT = [ROOT.TH1F("hsHT_{0}".format(j), "HT_{0}".format(j), 200,0,1000)
         for j in range(len(s))]

hsPt0 = [ROOT.TH1F("hsJ_Pt0_{0}".format(j), "J_Pt0_{0}".format(j),1000,0,2000)
         for j in range(len(s))]

hsPt1 = [ROOT.TH1F("hsJ_Pt1_{0}".format(j), "J_Pt1_{0}".format(j),1000,0,2000)
         for j in range(len(s))]

hsJEta0 = [ROOT.TH1F("hsJ_Eta0_{0}".format(j), "J_Eta0_{0}".format(j),50,0,10)
           for j in range(len(s))]

hsJEta1 = [ROOT.TH1F("hsJ_Eta1_{0}".format(j), "J_Eta1_{0}".format(j),50,0,10)
           for j in range(len(s))]

hsDeltaPhi = [ROOT.TH1F("hsDeltaPhi_{0}".format(j), "delta_Phi_{0}".format(j), 50, -1, 4)
              for j in range(len(s))]

# Highest invariant mass between to jets
hsMassMax = [ROOT.TH1F("hsMjj_{0}".format(j), "Max_M_{0}".format(j), 65,0,4000)
            for j in range(len(s))]

hsDeltaPhiMetJetMin = [ROOT.TH1F("hsDeltaPhiMin_{0}".format(j), "Min_delta_phi_{0}".format(j), 50, -1, 4)
                       for j in range(len(s))]

hsDeltaEtaJet = [ROOT.TH1F("hsDeltaEtaJet_{0}".format(j), "Delta_Eta_Jet12_{0}".format(j), 50,0,10)
                for j in range(len(s))]


# hsDeltaPhiMetJet = [[ROOT.TH1F("hsDelPhiMet_j{0}_{1}".format(i,j),"dMetJ{0}_{1}".format(i,j), 50, -1, 4) 
#                      for i in range(numJets)] 
#                     for j in range(len(s))]

#---------------------------
#        background
#---------------------------
# hbDeltaEtaMaxIM = ROOT.TH1F("hbDeltaEtaMaxIM", "Delta_Eta_MaxIM", 50,0,10)
# hbMass = ROOT.TH1F("hbMass", "M(j1,j2)", 65,0,4000)

hbHT = ROOT.TH1F("hbHT", "HT", 200,0,1000)
hbPt0 = ROOT.TH1F("hsJ_Pt0".format(j), "J_Pt0",1000,0,2000)
hbPt1 = ROOT.TH1F("hsJ_Pt1".format(j), "J_Pt1",1000,0,2000)
hbJEta0 = ROOT.TH1F("hbJ_Eta0}", "J_Eta0",50,0,10)
hbJEta1 = ROOT.TH1F("hbJ_Eta1}", "J_Eta1",50,0,10)
hbDeltaPhi = ROOT.TH1F("hbDeltaPhi", "delta_Phi", 50, -1, 4)

# Highest invariant mass between to jets
hbMassMax = ROOT.TH1F("hbMjj", "Max_M", 65,0,4000)

hbDeltaPhiMetJetMin = ROOT.TH1F("hsDeltaPhiMin", "Min_delta_phi", 50, -1, 4)
hbDeltaEtaJet = ROOT.TH1F("hbDeltaEtaJet", "Delta_Eta_Jet12", 50,0,10)

# hbDeltaPhiMetJet = [ROOT.TH1F("hbDelPhiMet_j{0}".format(i),"dMetJ{0}".format(i), 50, -1, 4) 
#                     for i in range(numJets)]

In [ ]:
def ht(jet, njets):
    h_t = 0
    for n in range(njets):
        h_t = h_t + abs(jet.At(n).PT)
    
    return h_t
    

### Creating dictionaries for the cuts

In [ ]:
s_c = [{ 'cut0':0, 'cut1':0, 'cut2':0, 'cut3':0, 'cut4':0, 'cut5':0, 'cut6':0, 'cut7':0, 'cut8':0}]*len(s)
b_c = { 'cut0':0, 'cut1':0, 'cut2':0, 'cut3':0, 'cut4':0, 'cut5':0, 'cut6':0, 'cut7':0, 'cut8':0}
c_c = { 'cut0': r'$N^o Jets > 2$',
        'cut1': r'$H_T > 200$',        
        'cut2': r'$P_T(J_i)>30\ i=0,1$',
        'cut3': r'$\eta(J_i)<5\ i=0,1$',
        'cut4': r'$\eta (J_0)*\eta (J_1) < 0$',
        'cut5': r'$|\Delta \phi (J_0,J_1)| > 2.3$',
        'cut6': r'$max(m(J_i,J_j)) > 1000$',
        'cut7': r'$min(|\Delta\phi(MET,J_i)|) > 0.5,\ i=0...4$',        
        'cut8': r'$|\Delta\eta(J_0,J_1)|<2.5$'}

### Selecting events and filling histograms

In [ ]:
# Loop over all signal events
for k in range(len(ns)):
    print(k)
    for event in range(ns[k]):
        #Load selected branches with data from specified event
        treeS[k].ReadEntry(event)
        
        NJets = jetS[k].GetEntries() if jetS[k].GetEntries() <= numJets else numJets

        # cuts
        if jetS[k].GetEntries() < 2: continue
        s_c[k]['cut0'] += 1
            
        if ht(jetS[k],NJets) < 200: continue
        s_c[k]['cut1'] += 1
            
        if jetS[k].At(0).PT < 30: continue
        if jetS[k].At(1).PT < 30: continue
        if abs(jetS[k].At(0).Eta) > 5: continue
        if abs(jetS[k].At(1).Eta) > 5: continue 
        if (jetS[k].At(0).Eta * jetS[k].At(1).Eta) > 0: continue
        s_c[k]['cut2'] += 1
        s_c[k]['cut3'] += 1
        s_c[k]['cut4'] += 1
            
        if abs(DeltaPhi(jetS[k].At(0).Phi,jetS[k].At(1).Phi)) < 2.3: continue
        s_c[k]['cut5'] += 1
            
        i = 0
        j = 0
        JsMass = 0
        
        dPhiMetJet = []
        
        for n in range(NJets):
            # selecting the two jet system 
            # with the largest invariant mass
            dPhiMetJet.append(abs(DeltaPhi(metS[k].At(0).Phi,jetS[k].At(n).Phi)))
            for m in range(NJets):
                if n==m:continue
                a = InvariantMass(jetS[k],n,m) 
                if a > JsMass:
                    i,j = m,n
                    JsMass = a
                    
        if JsMass < 1000: continue
        s_c[k]['cut6'] += 1
            
        if min(dPhiMetJet) < 0.5: continue
        s_c[k]['cut7'] += 1
        
        if abs(jetS[k].At(0).Eta-jetS[k].At(1).Eta) > 2.5: continue
        s_c[k]['cut8'] += 1
        
        hsHT[k].Fill(ht(jetS[k],NJets))
        hsPt0[k].Fill(jetS[k].At(0).PT)
        hsPt1[k].Fill(jetS[k].At(1).PT)
        hsJEta0[k].Fill(abs(jetS[k].At(0).Eta))
        hsJEta1[k].Fill(abs(jetS[k].At(1).Eta))
        hsDeltaPhi[k].Fill(abs(DeltaPhi(jetS[k].At(0).Phi,jetS[k].At(1).Phi)))
        hsMassMax[k].Fill(JsMass)
        hsDeltaPhiMetJetMin[k].Fill(min(dPhiMetJet))
        hsDeltaEtaJet[k].Fill(abs(jetS[k].At(0).Eta-jetS[k].At(1).Eta))
        
#         hsDeltaEtaMaxIM[k].Fill(abs(jetS[k].At(i).Eta-jetS[k].At(j).Eta))
#         hsMass[k].Fill(InvariantMass(jetS[k],0,1))
#         

#         hsDeltaPhiMax[k].Fill(abs(DeltaPhi(jetS[k].At(i).Phi,jetS[k].At(j).Phi)))

In [ ]:
# Loop over all background events 
for event in range(nb):
    # load selected branches with data from specified event
    treeB.ReadEntry(event)
    
    NJets = jetB.GetEntries() if jetB.GetEntries() <= numJets else numJets
    
    # cuts
    if jetB.GetEntries() < 2: continue
    b_c['cut0'] += 1
        
    if ht(jetB,NJets) < 200: continue
    b_c['cut1'] += 1
        
    if jetB.At(0).PT < 30: continue
    if jetB.At(1).PT < 30: continue
    if abs(jetB.At(0).Eta) > 5: continue
    if abs(jetB.At(1).Eta) > 5: continue 
    if (jetB.At(0).Eta * jetB.At(1).Eta) > 0: continue
    b_c['cut2'] += 1
    b_c['cut3'] += 1
    b_c['cut4'] += 1
        
    if abs(DeltaPhi(jetB.At(0).Phi,jetB.At(1).Phi)) < 2.3: continue
    b_c['cut5'] += 1
        
    i = 0
    j = 0
    JbMass = 0
    dPhiMetJet = []
    
    for n in range(NJets):
        # selecting the two jet system 
        # with the largest invariant mass
        dPhiMetJet.append(abs(DeltaPhi(metB.At(0).Phi,jetB.At(n).Phi)))
        for m in range(NJets):
            if n==m:continue
            a = InvariantMass(jetB,n,m) 
            if a > JbMass:
                i,j = m,n
                JbMass = a
                
    if JbMass < 1000: continue
    b_c['cut6'] += 1
        
    if min(dPhiMetJet) < 0.5: continue
    b_c['cut7'] += 1
        
    if abs(jetB.At(0).Eta-jetB.At(1).Eta) > 2.5: continue
    b_c['cut8'] += 1
    
    hbHT.Fill(ht(jetB,NJets))
    hbPt0.Fill(jetB.At(0).PT)
    hbPt1.Fill(jetB.At(1).PT)
    hbJEta0.Fill(abs(jetB.At(0).Eta))
    hbJEta1.Fill(abs(jetB.At(1).Eta))
    hbDeltaPhi.Fill(abs(DeltaPhi(jetB.At(0).Phi,jetB.At(1).Phi)))
    hbMassMax.Fill(JbMass)
    hbDeltaPhiMetJetMin.Fill(min(dPhiMetJet))
    hbDeltaEtaJet.Fill(abs(jetB.At(0).Eta-jetB.At(1).Eta))
    
#     hbDeltaEtaMaxIM.Fill((abs(jetB.At(i).Eta-jetB.At(j).Eta)))
#     hbMass.Fill(InvariantMass(jetB,0,1))

#     hbDeltaPhiMax.Fill(abs(DeltaPhi(jetB.At(i).Phi,jetB.At(j).Phi)))

In [ ]:
for i in range(len(s)):
    print(sNames[i])
    print(s_c[i])
print("background")
print(b_c)

In [ ]:
def pH(hBckg, hSig, plotName, sigLabels, savePath='./'):
    canvas = ROOT.TCanvas()
    canvas.cd()

    hBckg.Scale(1 / hBckg.Integral())
    hBckg.SetStats(0)
    hBckg.SetLineColor(2)

    for i in range(len(hSig)):
        hSig[i].Scale(1/hSig[i].Integral())
        hSig[i].SetStats(0)
        hSig[i].SetLineColor(3+i)



    hSig[0].SetMaximum(MaxVal(hBckg, hSig))
    hSig[0].Draw("pe")
    for i in range(1, len(hSig)):
        hSig[i].Draw("h,same")
    hBckg.Draw("h,same")

    legend = ROOT.TLegend(0.6,0.8,0.9,0.9)
    legend.AddEntry(hBckg, "Background")
    for i in range(len(hSig)):
        legend.AddEntry(hSig[i], sigLabels[i])
    legend.Draw()

    canvas.SaveAs(os.path.join(savePath, plotName + ".png"))
    canvas.SaveAs(os.path.join(savePath, plotName + ".c"))
    
pH(hbDeltaEtaJet,hsDeltaEtaJet,"7_delta_eta_j0_j2",sNames,imgPath)

   